<a href="https://colab.research.google.com/github/thaisneubauer/PPAR2021/blob/main/BPI_Challenge_2012_02RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inicialização do ambiente

## Importando as bibliotecas

In [ ]:
import sys
import csv
import math
import time
import random
import numpy as np
import pandas as pd
import plotly.express as px
from google.colab import files
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

In [ ]:
pip install neupy

     |████████████████████████████████| 235kB 25.0MB/s 
     |████████████████████████████████| 92.7MB 35kB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 368kB 36.2MB/s 
     |████████████████████████████████| 3.2MB 38.3MB/s 
ERROR: kapre 0.3.5 has requirement tensorflow>=2.0.0, but you'll have tensorflow 1.13.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: progressbar2 3.38.0
    Uni

In [ ]:
import numpy as np
from neupy import algorithms
from neupy.layers import *

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/usr/local/lib/p

## Conectando com o Drive 

In [ ]:
#para salvar os resultados no Drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


## Instalando biblioteca para tempo de execução

In [ ]:
!pip install ipython-autotime

%load_ext autotime

time: 172 µs (started: 2021-07-14 01:14:42 +00:00)


# Carregar os dados

In [ ]:
from google_drive_downloader import GoogleDriveDownloader as gdd

google_id = '1DgYTvfbzTe8CPMOwXMupmTOiMTP56PwD'
gdd.download_file_from_google_drive(file_id=google_id, 
                                    dest_path = './data.csv', 
                                    showsize = True)
data = pd.read_csv("data.csv")

30.3 MiB Done.
time: 5.16 s (started: 2021-07-14 01:14:42 +00:00)


In [ ]:
data.head()

,Case ID,Activity,Resource,Workload,Duration(s)
0,173712,W_Fixing incoming lead,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",109.0
1,173709,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",25.0
2,173703,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",158.0
3,173709,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'FREE', ...",142.0
4,173703,W_Filling in information for the application,10912,"{10912: 'FREE', 10982: 'FREE', 11019: 'HIGH', ...",1306.0


time: 81.6 ms (started: 2021-07-14 01:14:48 +00:00)


In [ ]:
len(data['Case ID'].unique())

6129

time: 8.34 ms (started: 2021-07-14 01:14:48 +00:00)


In [ ]:
len(data['Activity'].unique())

6

time: 20.1 ms (started: 2021-07-14 01:14:48 +00:00)


# Modelagem

## Definição dos elementos e análise de quantidades

- Estado: Activity + Resources Workload

- Ação: Resource

- Custo: Duration

Quantos estados diferentes podemos ter?

\#resources (do notebook de geração do dataset)

x 

\#níveis de worload que as resources podem apresentar (FREE, LOW, HIGH)

x 

\#atividades


In [ ]:
57*3*len(data['Activity'].unique())

1026

time: 18.1 ms (started: 2021-07-14 01:14:48 +00:00)


Portanto, são 1026 estados possíveis no contexto do problema se considerarmos todas as atividades, com todos os níveis de workload para todos os recursos.

In [ ]:
1026 * 57

58482

time: 5.4 ms (started: 2021-07-14 01:14:48 +00:00)


E 58 mil combinações possíveis de todos os estados com todas as ações disponíveis.

## Seleção das amostras

E se selecionarmos menos recursos?
Vou selecionar uma parte, como nos artigos:

In [ ]:
activity_count = pd.DataFrame(data.groupby('Resource')['Activity'].count().sort_values()).reset_index()

time: 21.1 ms (started: 2021-07-14 01:14:48 +00:00)


In [ ]:
px.box(activity_count, y = 'Activity')

time: 1.56 s (started: 2021-07-14 01:14:48 +00:00)


In [ ]:
#resources = list(data.Resource.unique())
#random.seed(42)
#random.shuffle(resources)
#resources2del = resources[30:]
resources2del = activity_count[activity_count.Activity < activity_count.Activity.mean()].Resource
print('#recursos desconsiderados:', len(resources2del))
cases2del = data[data.Resource.isin(resources2del)]['Case ID'].unique()
data_selected_resources = data[~data['Case ID'].isin(cases2del)]
print('#casos filtrando recursos:', len(data_selected_resources['Case ID'].unique()))

#recursos desconsiderados: 27
#casos filtrando recursos: 3480
time: 30.4 ms (started: 2021-07-14 01:14:49 +00:00)


In [ ]:
print('#casos sem filtrar recursos:', len(data['Case ID'].unique()))

#casos sem filtrar recursos: 6129
time: 3.34 ms (started: 2021-07-14 01:14:49 +00:00)


É necessário atualizar a lista de workload para que conste somente o workload desses recursos selecionados:

In [ ]:
import ast

def select_workload(w):
  w_d = ast.literal_eval(w)
  return str({res: w_d[res] for res in w_d if res not in resources2del})
data_selected_resources['Workload'] = data_selected_resources['Workload'].apply(select_workload).astype(str)

time: 3.62 s (started: 2021-07-14 01:14:49 +00:00)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
print('#workloads possíveis:',(57-27)*3)
print('#estados posssíveis (workloads x atividades):',(57-27)*3*len(data['Activity'].unique()))
print('#combinações possíveis de estado-ações:',(57-27)*3*len(data['Activity'].unique())*(57-27))

#workloads possíveis: 90
#estados posssíveis (workloads x atividades): 540
#combinações possíveis de estado-ações: 16200
time: 16.4 ms (started: 2021-07-14 01:14:53 +00:00)


## Geração do histórico H 

Formato *State*, *Action*, *Cost*

### Do conjunto completo de amostras

In [ ]:
H = pd.DataFrame()
H['case'] = data['Case ID']
H['state'] = data['Activity'] + "-" + data['Workload']
H['action'] = data['Resource']
H['cost'] = data['Duration(s)']
H.head()

,case,state,action,cost
0,173712,"W_Fixing incoming lead-{10912: 'FREE', 10982: ...",10912,109.0
1,173709,W_Filling in information for the application-{...,10912,25.0
2,173703,W_Filling in information for the application-{...,10912,158.0
3,173709,W_Filling in information for the application-{...,10912,142.0
4,173703,W_Filling in information for the application-{...,10912,1306.0


time: 53.9 ms (started: 2021-07-14 01:14:53 +00:00)


In [ ]:
H_original = H.copy()

time: 3.69 ms (started: 2021-07-14 01:14:53 +00:00)


### Do conjunto de amostras selecionadas

In [ ]:
len(data_selected_resources['Workload'].unique())

4783

time: 26.4 ms (started: 2021-07-14 01:14:53 +00:00)


In [ ]:
len(data_selected_resources['Activity'].unique())

6

time: 6.09 ms (started: 2021-07-14 01:14:53 +00:00)


In [ ]:
len(data_selected_resources['Resource'].unique())

30

time: 9.1 ms (started: 2021-07-14 01:14:53 +00:00)


In [ ]:
H_selected_resources = pd.DataFrame()
H_selected_resources['case'] = data_selected_resources['Case ID']
H_selected_resources['state'] = data_selected_resources['Activity'] + "-" + data_selected_resources['Workload']
H_selected_resources['action'] = data_selected_resources['Resource']
H_selected_resources['cost'] = data_selected_resources['Duration(s)']
H_selected_resources.head()

,case,state,action,cost
8,173961,"W_Fixing incoming lead-{10912: 'FREE', 10982: ...",10889,286.0
10,173781,W_Filling in information for the application-{...,11180,622.0
12,173934,"W_Fixing incoming lead-{10912: 'FREE', 10982: ...",10889,305.0
15,173901,"W_Fixing incoming lead-{10912: 'FREE', 10982: ...",10889,174.0
16,173904,"W_Fixing incoming lead-{10912: 'FREE', 10982: ...",10889,351.0


time: 38.9 ms (started: 2021-07-14 01:14:53 +00:00)


In [ ]:
H_selected_resources.to_csv('h_selected_resources.csv',index=False)
files.download('h_selected_resources.csv') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

time: 275 ms (started: 2021-07-14 01:14:53 +00:00)


In [ ]:
len(H_selected_resources.state.unique())

6364

time: 30.1 ms (started: 2021-07-14 01:14:54 +00:00)


## Analisando H

### Do conjunto completo de amostras

Quantos estados diferentes temos?

In [ ]:
len(H['state'].unique())

16902

time: 76.3 ms (started: 2021-07-14 01:14:54 +00:00)


Quantos recursos diferentes?

In [ ]:
len(H['action'].unique())

57

time: 5.33 ms (started: 2021-07-14 01:14:54 +00:00)


Qual o máximo de combinações state-action podemos ter, então?

In [ ]:
len(H['state'].unique())*len(H['action'].unique())

963414

time: 84.8 ms (started: 2021-07-14 01:14:54 +00:00)


### Do conjunto de amostras selecionadas

In [ ]:
len(H_selected_resources['state'].unique())

6364

time: 32.7 ms (started: 2021-07-14 01:14:54 +00:00)


In [ ]:
len(H_selected_resources['action'].unique())

30

time: 7 ms (started: 2021-07-14 01:14:54 +00:00)


In [ ]:
len(H_selected_resources['state'].unique())*len(H_selected_resources['action'].unique())

190920

time: 34.1 ms (started: 2021-07-14 01:14:54 +00:00)


# RL

## Pré-definições e análises

Definição do parâmetro MAX_DIFF - um dos critérios de parada:

In [1]:
px.box(H_selected_resources, y='cost') # para escolher o MAXDIFF - Q3

NameError: ignored

In [ ]:
H_selected_resources.cost.quantile(.25)

Para cada ação (recurso) em cada estado, o próximo estado pode ser diferente (processo estocástico) e mesmo para um mesmo próximo estado, o custo pode ser diferentes. E qual custo usar? Será que há diferença entre o custo médio, mínimo e máximo pra cada tupla estado-ação-próximo_estado?

In [ ]:
#pd.DataFrame(Q.groupby(['state','action','next_state'])['cost'].agg(['mean','max','min'])).round(2)

Como podemos ver nos resultados exibidos para as duas primeiras linhas, há diferenças. Vamos usar a média como um primeiro teste.

## Algoritmo

In [ ]:
ALG = 'FQI'
GAMMA = 0.9
MAX_IT = 100
MAX_DIFF = 0
MAX_TIME = 10*60*60 #treinamento de no max 10h
print('MAX_DIFF =', MAX_DIFF)

def insert_next_s(case):
  case['next_state'] = case['state'].shift(-1)
  case = case.fillna('END')
  return case

def get_y(x, n, q, gamma):#, scale):
  q_next_s = 0
  if x['next_state'] != 'END':
    q_next_s = q[q['state']==x['next_state']].groupby('action')[n-1].mean().min()
  #if scale:
  #  scaler = StandardScaler().fit(q_next_s)
  #  return scaler, scaler.transform(x['cost'] + gamma * q_next_s)
  #return None, x['cost'] + gamma * q_next_s
  return x['cost'] + gamma * q_next_s

#def set_q(pred_values, scaler):
#  if scaler is not None:
#    return scaler.inverse_transform(pred_values)
#  return pred_values

def train(model, x_train, q, max_it = MAX_IT, max_diff = MAX_DIFF, gamma = GAMMA):#, scale = False):
  start_time = time.time()
  for n in range(1,max_it):
    print(n)
    y = q.apply(lambda x: get_y(x, n, q, gamma), axis=1)
    model = model.fit(x_train, y)
    #model.fit(x_train, y)
    q[n] = model.predict(x_train)
    if (q[n-1] - q[n]).sum() < max_diff or (time.time()-start_time)>MAX_TIME: 
      break
  return q, n, time.time()-start_time

MAX_DIFF = 0
time: 42.6 ms (started: 2021-07-14 01:14:54 +00:00)


## Inicialização

In [ ]:
def init_q(h):
  aux = h.groupby('case').apply(insert_next_s)
  aux[0] = h.cost.max()
  return aux

time: 1.84 ms (started: 2021-07-14 02:05:22 +00:00)


In [ ]:
h = H_selected_resources.copy()
MAX_DIFF = H.cost.quantile(.25)
q = init_q(h)
q['action'] = q['action'].astype(str) # para que o get_dummies funcione corretamente para essa coluna
x = pd.get_dummies(q[['state','action']])
print('#linhas (datapoints), #colunas (features):', x.shape)


#linhas (datapoints), #colunas (features): (12101, 6394)
time: 8.32 s (started: 2021-07-14 02:05:23 +00:00)


In [ ]:
h.cost.max()

243191.0

time: 6.9 ms (started: 2021-07-14 01:15:02 +00:00)


## Execuções

In [ ]:
def save_exec(alg, alg_espec, model_type, model_espec, gamma, it, train_time, q):
  #save metadata
  exec_metadata = [alg, alg_espec, model_type, model_espec, gamma, it, train_time]
  with open(r'/gdrive/My Drive/ppar_results/metadata.csv','a') as f:
    writer = csv.writer(f)
    writer.writerow(exec_metadata)
  #save q
  q_filename = ('-').join([str(info) for info in exec_metadata[:-2]])
  with open('/gdrive/My Drive/ppar_results/%s.csv'%q_filename, 'w') as f:
    q.to_csv(f, index=False)

time: 11 ms (started: 2021-07-14 01:15:02 +00:00)


#### FQI - LinearRegression

In [ ]:
q, it, train_time = train(LinearRegression(), x, init_q(h))
save_exec(ALG, 'none', 'linear_regression', 'default', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
time: 1h 24min 26s (started: 2021-07-14 02:05:32 +00:00)


#### FQI - Random Forest

In [ ]:
#q, it, train_time = train(RandomForestRegressor(n_estimators=5), x, init_q(h))
#save_exec(ALG, 'none', 'random_forest', 'n_estimators=5', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
time: 1h 11min 42s (started: 2021-07-08 20:22:50 +00:00)


In [ ]:
#q, it, train_time = train(RandomForestRegressor(n_estimators=10), x, init_q(h))
#save_exec(ALG, 'none', 'random_forest', 'n_estimators=10', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
time: 2h 15min 51s (started: 2021-07-05 15:03:24 +00:00)


In [ ]:
#q, it, train_time = train(RandomForestRegressor(n_estimators=20), x, init_q(h))
#save_exec(ALG, 'none', 'random_forest', 'n_estimators=20', GAMMA, it, train_time, q)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
time: 4h 34min 20s (started: 2021-07-07 19:47:11 +00:00)


In [ ]:
#q, it, train_time = train(RandomForestRegressor(n_estimators=50), x, init_q(h))
#save_exec(ALG, 'none', 'random_forest', 'n_estimators=20', GAMMA, it, train_time, q)

1
2
3
4
5
6
7


In [ ]:
#q, it, train_time = train(RandomForestRegressor(), x, init_q(h)), max_it=3)
#save_exec(ALG, 'none', 'random_forest', 'default', GAMMA, it, train_time, q)

1
2
time: 2h 30min 27s (started: 2021-07-06 01:30:15 +00:00)


#### NFQ - MLP

In [ ]:
#q, it, train_time = train(MLPRegressor(learning_rate='adaptive', alpha=0.005, hidden_layer_sizes=(10,10)), x, init_q(h))
#save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, alpha=0.005, hidden_layer_sizes=(10,10)', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3


...

23


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



24


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 1h 36min 55s (started: 2021-07-09 03:35:20 +00:00)


In [ ]:
#q, it, train_time = train(MLPRegressor(learning_rate='adaptive', alpha=0.005, hidden_layer_sizes=(50,50)), x, init_q(h))
#save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, alpha=0.005, hidden_layer_sizes=(50,50)', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3


...

17


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



18


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 1h 57min 14s (started: 2021-07-09 05:53:03 +00:00)


In [ ]:
# Mesmo com max_iter=1000, nao converge
#q, it, train_time = train(MLPRegressor(learning_rate='adaptive', max_iter = 1000), x, init_q(h))
#save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive', GAMMA, it, train_time, q)

time: 1.34 ms (started: 2021-07-12 05:30:05 +00:00)


In [ ]:
#q, it, train_time = train(MLPRegressor(learning_rate = 'adaptive', activation = 'logistic'), x, init_q(h))
#save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, activation=logistic', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3


...

7


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



8


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 1h 47s (started: 2021-07-12 02:36:08 +00:00)


In [ ]:
#q, it, train_time = train(MLPRegressor(learning_rate = 'adaptive', activation = 'logistic', hidden_layer_sizes=(50,50)), x, init_q(h))
#save_exec(ALG, 'none', 'mlp', 'learning_rate=adaptive, activation=logistic, hidden_layers=(50,50)', GAMMA, it, train_time, q)

1


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



2


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



3


...

4


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



5


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning:

Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.



time: 29min 51s (started: 2021-07-12 05:30:52 +00:00)


#### NFQ - RProp

In [ ]:
#Pra comparar com as primeiras execucoes de MLP, que usa Relu por padrao
#network = Input(x.shape[1]) >> Relu(50) >> Relu(50) >> Relu(1)
#q, it, train_time = train(algorithms.RPROP(network), x, init_q(h))
#save_exec(ALG, 'none', 'RPROP', '50,50_relu', GAMMA, it, train_time, q)

1


2


3


4


5


...

34


35


36


37


time: 48min 22s (started: 2021-07-12 18:05:42 +00:00)


In [ ]:
#network = Input(x.shape[1]) >> Sigmoid(500) >> Sigmoid(500) >> Sigmoid(1)
#q, it, train_time = train(algorithms.RPROP(network), x, init_q(h))
#save_exec(ALG, 'none', 'RPROP', '500,500-Sigmoid', GAMMA, it, train_time, q)

Instructions for updating:
Colocations handled automatically by placer.
1


2


time: 16min 47s (started: 2021-07-12 05:13:18 +00:00)


In [ ]:
#network = Input(x.shape[1]) >> Sigmoid(100) >> Sigmoid(1)
#q, it, train_time = train(algorithms.RPROP(network), x, init_q(h))
#save_exec(ALG, 'none', 'RPROP', '100-Sigmoid', GAMMA, it, train_time, q)